In [1]:
!pip install pyspark

     |████████████████████████████████| 316.9 MB 76 kB/s  eta 0:00:013   |███▎                            | 33.0 MB 119 kB/s eta 0:39:29     |█████▊                          | 57.1 MB 126 kB/s eta 0:34:07     |███████████▎                    | 111.5 MB 15.1 MB/s eta 0:00:14     |██████████████                  | 139.3 MB 15.5 MB/s eta 0:00:12     |███████████████▌                | 152.9 MB 10.6 MB/s eta 0:00:16     |███████████████████████▎        | 230.2 MB 4.1 MB/s eta 0:00:22     |█████████████████████████▎      | 250.6 MB 5.1 MB/s eta 0:00:14     |█████████████████████████▍      | 251.5 MB 5.1 MB/s eta 0:00:13
     |████████████████████████████████| 200 kB 16.9 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425365 sha256=a1478bc8530c87e8665a573154de6f76712a201c9a4b623e21a8064bfb653dd5
  Stored in directory: /Users/ankitrajvanshi/Library/Caches/pip/wheels/57/bd/14/ce9e21f2649298678d011fb8f71ed38ee70b42b94fef0be142
Successfully built

In [5]:
!pip install 'pyspark[sql]'

     |████████████████████████████████| 26.9 MB 8.4 MB/s eta 0:00:01


In [6]:
!pip install 'pyspark[pandas_on_spark]' plotly

In [7]:
!pip install 'pyspark[connect]'

     |████████████████████████████████| 230 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 9.6 MB 25.3 MB/s eta 0:00:01     |█████████                       | 2.7 MB 25.3 MB/s eta 0:00:01
     |████████████████████████████████| 394 kB 5.7 MB/s eta 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.1
    Uninstalling protobuf-3.19.1:
      Successfully uninstalled protobuf-3.19.1
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.42.0
    Uninstalling grpcio-1.42.0:
      Successfully uninstalled grpcio-1.42.0
  Attempting uninstall: googleapis-common-protos
    Found existing installation: googleapis-common-protos 1.53.0
    Uninstalling googleapis-common-protos-1.53.0:
      Successfully uninstalled googleapis-common-protos-1.53.0


In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 00:20:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [41]:
from pyspark.sql.functions import col, regexp_replace, split

In [49]:
df = spark.read.csv("movies.dat", sep='::', schema='MovieID int, oldTitle string, combinedGenres string')

In [52]:
# Split title_year column into title, year
df_modified = df.withColumn("oldTitle_Modified", regexp_replace("oldTitle", "[)]", "("))
split_df = df_modified.withColumn("Title_Year", split("oldTitle_Modified", '\\('))
final_df = split_df.withColumn("Title", col("Title_Year")[0]).withColumn("Year", col("Title_Year")[1]).withColumn("Genres", split("combinedGenres", '\\|'))
final_df = final_df.drop("oldTitle", "oldTitle_Modified", "Title_Year", "combinedGenres")
final_df.show()
final_df.write.json("movies.json")

+-------+--------------------+----+--------------------+
|MovieID|               Title|Year|              Genres|
+-------+--------------------+----+--------------------+
|      1|          Toy Story |1995|[Animation, Child...|
|      2|            Jumanji |1995|[Adventure, Child...|
|      3|   Grumpier Old Men |1995|   [Comedy, Romance]|
|      4|  Waiting to Exhale |1995|     [Comedy, Drama]|
|      5|Father of the Bri...|1995|            [Comedy]|
|      6|               Heat |1995|[Action, Crime, T...|
|      7|            Sabrina |1995|   [Comedy, Romance]|
|      8|       Tom and Huck |1995|[Adventure, Child...|
|      9|       Sudden Death |1995|            [Action]|
|     10|          GoldenEye |1995|[Action, Adventur...|
|     11|American Presiden...|1995|[Comedy, Drama, R...|
|     12|Dracula: Dead and...|1995|    [Comedy, Horror]|
|     13|              Balto |1995|[Animation, Child...|
|     14|              Nixon |1995|             [Drama]|
|     15|   Cutthroat Island |1

In [58]:
df = spark.read.csv('ratings.dat', sep='::', schema='UserID int, MovieID int, Rating int, Timestamp long')
df_single_partition = df.coalesce(1)
df_single_partition.write.csv("ratings.csv", header=True, mode="overwrite")
df.show()

+------+-------+------+---------+
|UserID|MovieID|Rating|Timestamp|
+------+-------+------+---------+
|     1|   1193|     5|978300760|
|     1|    661|     3|978302109|
|     1|    914|     3|978301968|
|     1|   3408|     4|978300275|
|     1|   2355|     5|978824291|
|     1|   1197|     3|978302268|
|     1|   1287|     5|978302039|
|     1|   2804|     5|978300719|
|     1|    594|     4|978302268|
|     1|    919|     4|978301368|
|     1|    595|     5|978824268|
|     1|    938|     4|978301752|
|     1|   2398|     4|978302281|
|     1|   2918|     4|978302124|
|     1|   1035|     5|978301753|
|     1|   2791|     4|978302188|
|     1|   2687|     3|978824268|
|     1|   2018|     4|978301777|
|     1|   3105|     5|978301713|
|     1|   2797|     4|978302039|
+------+-------+------+---------+
only showing top 20 rows



In [56]:
df = spark.read.csv('users.dat', sep='::', schema='UserID int, Gender string, Age int, Occupation int, Zipcode int')

df.show()
df.write.json("users.json")

+------+------+---+----------+-------+
|UserID|Gender|Age|Occupation|Zipcode|
+------+------+---+----------+-------+
|     1|     F|  1|        10|  48067|
|     2|     M| 56|        16|  70072|
|     3|     M| 25|        15|  55117|
|     4|     M| 45|         7|   2460|
|     5|     M| 25|        20|  55455|
|     6|     F| 50|         9|  55117|
|     7|     M| 35|         1|   6810|
|     8|     M| 25|        12|  11413|
|     9|     M| 25|        17|  61614|
|    10|     F| 35|         1|  95370|
|    11|     F| 25|         1|   4093|
|    12|     M| 25|        12|  32793|
|    13|     M| 45|         1|  93304|
|    14|     M| 35|         0|  60126|
|    15|     M| 25|         7|  22903|
|    16|     F| 35|         0|  20670|
|    17|     M| 50|         1|  95350|
|    18|     F| 18|         3|  95825|
|    19|     M|  1|        10|  48073|
|    20|     M| 25|        14|  55113|
+------+------+---+----------+-------+
only showing top 20 rows

